In [9]:
import pandas as pd
import datetime as dt
import missingno as msno
import matplotlib.pyplot as plt
from fuzzywuzzy import fuzz, process

ModuleNotFoundError: No module named 'missingno'

### Restrições de tipos de dados

É possível que alguma feature possui dados com um data type não usual, o que pode causar problemas de processamento.

In [ ]:
df.info() # verificar valores ausentes
df.dtypes # mostra o tipo de cada feature

In [ ]:
# números como string (exemplo: '57$')  -> e se o sifrão estiver no começo?
df['price'].sum() # retorna concatenação das strings

# converter:
df['price'] = df['price'].str.strip('$') # retira o caractere do exemplo
df['price'] = df['price'].astype('int') # converte

# verificar:
assert df['price'].dtype == 'int'

In [ ]:
# categórico como inteiro (exemplo: 0 para não, 1 para sim)
df['maried'].describe() # vai retornar média, desvio padrão etc.
    # era melhor dados estatísticos de frêquencia para categorias
    
# converter:
df['maried'] = df['maried'].astype('category')

In [ ]:
# data como objeto:
df.dtypes

# converter:
df['date'] = pd.to_datetime(df['date'])

# assert:
assert user_signups['subscription_date'].dtype == 'datetime64[ns]'

### Restrições de intervalo

Exemplo: 
- 6 estrelas em avaliação de 1 a 5;
- Registro em data no futuro.

Soluções:
- dropar o registro (não recomendado se temos muitos registros com o problema)
- setar mínimos e máximos e substituir
- tratar o dado como faltante e imputá-lo ()
- substituir por um valor fixo (como a média, mediana)

In [2]:
# filtrando datas futuras
import datetime as dt
df[df['date']>dt.date.today()]

# dropando datas futuras
today_date = dt.date.today()
# Drop values using filtering
df = df[df['subscription_date'] < today_date]
# Drop values using .drop()
df.drop(df[df['subscription_date'] > today_date].index, inplace = True)

# substituindo
# using filtering
df.loc[df['subscription_date'] > today_date, 'subscription_date'] = today_date
# Assert is true
assert df.subscription_date.max().date() <= today_date


In [ ]:
# dropando: filtrando e atribuindo
df = df[df['rating']<=5]

# dropando: método drop
df.drop(df[df['rating']>5].index, inplace = True) # inplace = true: método .drop modifica o objeto; se = False ele deve ser atribuido a outra variável.

# verificação:
assert df['rating'].max() <=5 

In [ ]:
# substituindo pelo limite:
df.loc[df['rating'] > 5, 'rating'] = 5 # com esse segundo argumento seleciona so a coluna 'rating'

# assert igual a anterior

### Registros duplicados

- Registros inteiros duplicados -> drope um deles

In [ ]:
# identificando
duplicates = df.duplicated() # booleano True para linhas duplicadas; False em todas as primeiras ocorrências
df[df.duplicated()] # retorna todas as linhas duplicadas

# argumentos do método .duplicated()
df2 = df[df.duplicated(subset = ['column1', 'column2'])]    # subset: especificamos quais colunas checar por duplicatas; recebe lista de nomes de features/colunas 
df2 = df[df.duplicated(keep = 'first')]    #keep: admite as strings 'first' (default), 'last' e False; mantém (retorna False para) a ocorrência especificada

# checando após modificação:
df[df.duplicated()].sorf_values(by = 'column1')

In [ ]:
# dropando duplicatas completos (registro inteiro é duplicado)

# método .drop_duplicates(): obs.:os argumentos são: subset, keep e inplace (iguais aos do .duplicated() e .drop())
df.drop_duplicates(inplace = True)

In [ ]:
# dropando duplicatas com divergências numéricas:

# Output duplicate values
column_names = ['first_name','last_name','address']
duplicates = df.duplicated(subset = column_names, keep = False)
df[duplicates].sort_values(by = 'first_name')

# decisão por estatística:encadeado método .groupby() e .agg()
# Group by column names and produce statistical summaries
column_names = ['first_name','last_name','address'] # colunas a agrupar (cujos valores serão duplicados)
summaries = {'height': 'max', 'weight': 'mean'} # colunas divergentes e o tipo de agregação (máximo e média)
df = df.groupby(by = column_names).agg(summaries).reset_index() # agrupa pela lista column_names; agrega pelo dict summaries
    # reset_index para criar novo índice após a agregação
# Make sure aggregation is done
duplicates = df.duplicated(subset = column_names, keep = False)
df[duplicates].sort_values(by = 'first_name')

### Membership constraints / restrições de associação

#### Dados categóricos

- Normalmente representados por um conjunto finito de inteiros e não admite outros valores;
- erros aqui costumam vir de campos de texto livre (caixa de texto) ou campos suspensos (campo com lista selecionável), ou ainda erro de parsing dos dados.

Temos 3 tipos de solução:
- droppar os valores;
- remapear as categorias;
- inferir os valores.

Boa prática: sempre que tivermos categorias, armazenar log (em dicionário ou dataframe) das categorias admitidas;

Daí, podemos fazer um (anti) joinning para retornar os dados do dataframe (esquerda) que não estão no log de categorias (direita). Método Difference


In [ ]:
# printa valores que não estão no log de categorias permitidas
inconsistent_categories = set(df['cat']).difference(df_log(['cat']))
print(inconsistent_categories)

# registros inconsistentes
inconsistent_rows = df['cat'].isin(inconsistent_categories)
inconsistent_data = df[inconsistent_rows]

# dropando
consistent_data = df[~inconsistent_data]

Inconsistência de valor:
Capitalização: quando se espera o valor 'single' e tem o valor 'Single' ou 'SINGLE'.

In [ ]:
# contando os valores categóricos:
df['cat'].value_counts() # só funciona em pandas series
df_cat.groupby('cat').count() # para dataframe podemos agrupar pela coluna

# tansformando a capitalização:
df['cat'] = df['cat'].str.upper()
df['cat'] = df['cat'].str.lower()

Espaços desnecessários (trailing spaces): 'single ' ou ' single'

In [ ]:
df['cat'] = df['cat'].str.strip() # default remove espaços à direita e esquerda.

##### Colapsando dados
Agrupando range de valores por categoria

exemplo: grupo de renda a partir do dado de renda

In [ ]:
# método qcut:
ranges = [0,200,500,np.inf]
group_names = ['0-200', '200-500','500+']

df['grupos'] = pd.cut(df['renda'],bins = ranges, labels = group_names) 

print(df[['grupos','renda']]) 


# existe o método qcut, voce nao define o range então ele categoriza na ordem
df['grupos'] = pd.cut(df['renda'],q = 3, labels = group_names) # q é o numero de grupos; classifica na ordem dividindo igualmente pra cada grupo

Redizir o número de categorias

In [ ]:
# usando dicionário de mapeamento e método replace:
map_dict = {} # chave é o antigo valor, definição é o novo valor

df['cat'] = df['cat'].replace(map_dict)


Limpando textos

In [ ]:
# substituição de caracteres
df['cat'] = df['cat'].str.replace("+", "00") # substitui mais por zero zero
df['cat'] = df['cat'].str.replace("-", "") # remove traços

# checagem
assert df['cat'].str.contains("+|-").any() == False # '|' é igual a OU

# eliminando strings com caracteres além do limite:
digits = df['cat'].str.len() # extraindo a quantidade de caracteres
df.loc[digits<10, 'cat'] = np.nan

# checagem
sanity_check = df['cat'].str.len()
assert sanity_check.min() >= 10

In [ ]:
# para limpezas mais complexas usamos regex
df['numbers'] = df['numbers'].str.replace(r'\D+',"") # removendo qualquer digito naõ numérico

# Uniformidade

Problemas: grandezas com unidades diferentes, datas com formatos diferentes, casas decimais existentes ou não.


In [ ]:
# unidades diferentes: 
# plote o scatter dos valores pra perceber o problema (escalas diferentes podem dar aglomerações diferentes)
# pesquise a fórmula e faça a conversão de unidade dos dados filtrados a partir de um limite (linha que divide os clusters)


# datas diferentes: 
# converta para datetime (pode causar erro caso haja datas impossíveis)
# então faça assim:
df['date'] = pd.to_datetime(df['date'],
                            infer_datetime_format=True, # tenta inferir formato da data
                            errors='coerce') # retorna na quando não consegue

# convertendo data para um formato específico:
df['date'] = df['date'].dt.strftime("%d-%m-%Y") 

# ainda assim, podemos ter dados impossíveis de inferir; ex.: 2019-03-08 é 8 de março ou 3 de agosto?
    # pode-se dropar o registro, tentar inferir de outros registros, chutar.

## Validação de campo crusado / cross field validation

Utilizar mais de um campo para validar o formato dos números.

Ex.: utilizar a soma dos acidentes graves e não graves para comparar com o total de acidentes em cada linha.
Ex.: hoje - data de aniversário == idade

In [ ]:
sum_classes = flights[['economy_class', 'business_class', 'first_class']].sum(axis = 1)
    # sum horizontally
passenger_equ = sum_classes == flights['total_passengers']
    # compare to total of the register
# Find and filter out rows with inconsistent passenger totals
inconsistent_pass = flights[~passenger_equ]
consistent_pass = flights[passenger_equ]


In [ ]:
import pandas as pd
import datetime as dt
# Convert to datetime and get today's date
users['Birthday'] = pd.to_datetime(users['Birthday'])
today = dt.date.today()
# For each row in the Birthday column, calculate year difference
age_manual = today.year - users['Birthday'].dt.year
# Find instances where ages match
age_equ = age_manual == users['Age']
# Find and filter out rows with inconsistent age
inconsistent_age = users[~age_equ]
consistent_age = users[age_equ]


Providências:
- dropar
- atribuir Na e tratar depois
- aplicar regras de conhecimento de  domínio

## Dados Faltantes

Pode ser representados por NaN (not a number), Na, 0, ...

In [ ]:
df.isna() # retorna True e False pra cada valor de célula.
df.isna().sum() # retorna o total de NaNs por coluna

# retornar dataframe filtrado por dados faltantes e não faltantes:
df[df['coluna'].isna()]  # registros com dados faltantes nessa coluna
df[~df['coluna'].isna()] # registros completos nessa coluna

In [ ]:
import missingno as msno
import matplotlib.pyplot as plt
# pacote útil para visualizar e entender dados faltantes

msno.matrix(df) # mostra a distribuição dos valores faltantes pra cada coluna (linha branca)
    # combinados com sort_values, podemos observar se há lógica nessa falta de dados:
        # ex.: ordenar o df por uma temperatura, 
        # podemos observar que para baixas temperaturas o sensor não consegue ler o nível de oxigênio no ar
plt.show()

#### Tipos de dados faltantes:

- MCAR (missing completely at random): sem relação sistemática entre dados faltantes e outros valores
    - ex.: erros de entrada de dados
- MAR (missing at random): com relação sistemática entre dados faltantes e outros valores observados
    - ex.: erros de leitura de ozônio a altas temperaturas
- MNAR (missing not at random): relação sistemática entre dados faltantes e variáveis não observadas
    - ex.: erros de letura de temperatura a altas temperaturas
    - Não há como classificar nesse tipo apenas observando os dados.

#### LIdando com dados faltantes:

- dropando o registro
- substituir por medida estatística (média, mediana, moda)
- Abordagens mais complexas:
    - imputando uma abordagem algorítmica
    - imputando com modelos de machine learning
    

In [ ]:
# dropando:
df_dropped = df.dropna(subset=['coluna'])

# substituindo:
coluna_mean = df['coluna'].mean()
df_replaced = df.fillna({'coluna': coluna_mean})


## Comparando strings
minimun edit distance / distância mínima de edição:
- maneira sistemática de identificar semelhança entre strings.
- Def.: mínimo número de passos necessários para transformar uma string em outra com as operações:
    - Inserção de caracteres, remoção, substituição e transposição.

Exemplo: INTENTION -> EXECUTION

INTENTION -> *NTENTION -> *NTECNTION -> *ETECNTION -> *EXECNTION -> *EXECUTION
<p>Tira I, inclui C, substitui N por E, T por X, N por U = 5 passos</p>

Para encontrar esses mínimos existem vários algoritmos que usam um ou mais dessas operações.
<p>Algorithm: Operations
<p>Damerau-Levenshtein: insertion,substitution,deletion,transposition
<p>Levenshtein: insertion,substitution,deletion
<p>Hamming: substitution only
<p>Jaro distance: transposition only
<p>......Possible packages:nltk,fuzzywuzzy,textdistance..


#### Comparando strings: similaridade
É um score de similaridade. Sendo 0: completamente diferentes e 100: identicos.
- usaremos o algoritmo Levenshtein (inserção, substituição, deleção) com o pacote fuzzywuzzy. Ele retorna um score de similaridade.


In [3]:
!pip install fuzzywuzzy


[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
# importa pacote
from fuzzywuzzy import fuzz

fuzz.WRatio('Reeding', 'Reading') # Reeding é erro de digitação

86

In [5]:
fuzz.WRatio('Los Angeles Lakers', 'Lakers') # Similaridade com palavra chave

90

In [6]:
fuzz.WRatio('Houston Rockets vs Los Angeles Lakers', 'Lakers vs Rockets') # ordenamento diferente

86

In [12]:
# COMPARAR UMA STRING COM UMA ARRAY DE STRINGS. 
from fuzzywuzzy import process

# Define string and array of possible matches
string = "Houston Rockets vs Los Angeles Lakers"
choices = pd.Series(['Rockets vs Lakers', 'Lakers vs Rockets', 'Houson vs Los Angeles', 'Heat vs Bulls'])

process.extract(string, choices, limit = 3)
# RETORNA <limit> TUPLAS DE ESTRUTURA (string, minimun edit distance, ordem de maior minimun edit distance para a menor)


[('Rockets vs Lakers', 86, 0),
 ('Lakers vs Rockets', 86, 1),
 ('Houson vs Los Angeles', 86, 2)]

In [ ]:
# colapsando em categorias strings similares
print(df['categoria'].unique()) # observe os valores únicos

# crie um dataframe com as strings que serão as categorias
categories = pd.DataFrame({'state': {0: 'California', 1: 'New York'}})

for state in categories['state']:
    # encontrar possíveis matches:
    matches = process.extract(state, df['categoria'], limit = df.shape[0]) # df.shape[0] é o número de linhas/categorias

    for potential_match in matches:
        # se a similaridade for alta (segundo termo da tupla >= 80)
        if potential_match[1] >=80:
            # substitui pela categoria:
            df.loc[df['categoria'] == potential_match[0], 'categoria'] = state

In [ ]:
unique_types = df['categoria'].unique()
print(process.extract('exemplo', unique_types, limit = len(unique_types)))
# observando este print, podemos estabelecer o valor limite do score para essa categoria.
# repetir o processo para os demais valores alvo da categoria

In [ ]:
# tudo junto:
# Iterate through categories
for cuisine in categories:  # categorias alvo
  # Create a list of matches, comparing cuisine with the cuisine_type column
  matches = process.extract(cuisine, restaurants['cuisine_type'], limit=len(restaurants.cuisine_type))

  # Iterate through the list of matches
  for match in matches:
     # Check whether the similarity score is greater than or equal to 80
    if match[1] >= 80:
      # If it is, select all rows where the cuisine_type is spelled this way, and set them to the correct cuisine
      restaurants.loc[restaurants['cuisine_type'] == match[0]] = cuisine
      
# Inspect the final result
print(restaurants['cuisine_type'].unique())

## Record linkage / ligação de registros
<p>Def.: ato de vincular dados de diferentes fontes em relação à mesma entidade. Seguindo as etapas:</p>

- Obter dados de diferentes fontes
- gerar pares (correspondencia entre as bases de dados)
- comparar os pares (gerar os scores)
- parear scores (selecionar limites)
- linkar os dados (merge, observando as regras de negócio para duplicatas)

- Condição ideal: comparar cada registro da base A com a base B (len(A)*len(B) registros, escala muito mal)
- Blocking: usar uma das informações de colunas para eliminar vários pares desnecessários


In [ ]:
# pacote
import recordlinkage

# criando o objeto: de indexação/ indexer
indexer = recordlinkage.Index()

# blocking (block method): gernando pares bloqueados na coluna 'coluna1'
indexer.block('coluna1') # qual a melhor coluna pra cá?
# gerando os pares:
pairs = indexer.index(database_A, database_B) 
    # o objeto resultante é um pandas dataframe com multi-index (índice em duas camadas)
    # cada camada de índice representa um índice de registro (??????)

# cria o objeto de comparação
compare_cl = recordlinkage.Compare()

# encontrar pares exatos para coluna1 e coluna2
compare_cl.exact('coluna1', 'coluna1', label='coluna1')
compare_cl.exact('coluna2', 'coluna2', label='coluna2')

# encontrar pares similares:
compare_cl.string('coluna3', 'coluna3', threshold=0.85, label='coluna3') #  threshold=0.85 é o limite inferior do score
compare_cl.string('coluna4', 'coluna4', threshold=0.85, label='coluna4')

# encontrando os matches:
potential_matches = compare_cl.compute(pairs, database_A, database_B)
    # o output é um pandas dataframe multi-index, cujo primeiro índice é o índice de linha do primeiro dataframe (database_A)
    #  o segundo índice é a lista de todos os índices de linha do segundo (database_B) para cada registro do primeiro.
    # as colunas são os valores comparados (coluna1 a coluna4, chamados nos métodos exact() e string())
    # os valores de célula (0 ou 1) indicam match de forma booleana.

# para encontrar potenciais matches: filtrar linhas que somadas estão acima de certo limite (hiperparâmetro ou parâmetro?) (ex.: 2)
n = 4 # procura matches em todas as colunas (duplicata)
n = 3 # match em 3 de 4 colunas já é sulficiente para declarar duplicata
matches = potential_matches[potential_matches.sum(axis=1) >= n]


In [ ]:
 # matches.index -> retorna o multi index em array 
duplicated_rows = matches.index.get_level_values(1) # qual camada do índice extrair = 1 (segunda camada); pode-se usar o nome da camada tbm

# encontrando duplicatas no dataframe 'database_B':
database_B_duplicates = database_B[database_B.index.isin(duplicated_rows)] # filtra por indice
# encontrando NÃO duplicatas no dataframe 'database_B':
non_dup_B = database_B[~database_B.index.isin(duplicated_rows)] # filtra por indice

full_database = database_A.append(non_dup_B)